In [72]:
import re
from collections import Counter
import numpy as np

In [18]:
def words(text):
    return re.findall(r'\b\w+\b',text.lower())

In [19]:
aspell_vocab=words(open('aspell.txt').read())
big_vocab=words(open('big.txt').read())
birkbeck_vocab=words(open('birkbeck.txt').read())
spell1_vocab=words(open('spell-testset1.txt').read())
spell2_vocab=words(open('spell-testset2.txt').read())

In [20]:
temp=aspell_vocab+big_vocab+birkbeck_vocab+spell1_vocab+spell2_vocab
VOCAB=set(temp)
counter_dic={}
counter_dic=Counter(temp)

In [27]:
print(f'the total word instances in all the texts is {len(temp)} and total word types is {len(VOCAB)} ')

the total word instances in all the texts is 1160441 and total word types is 65066 


In [80]:
def get_prob(counter_dic):
    '''
    Input:
        counter_dic: The wordcount dictionary where key is the word and value is its frequency.
    Output:
        probs: A dictionary where keys are the words and the values are the probability that a word will occur. 
    '''
    probs={}
    N=sum(counter_dic.values())
    for key in counter_dic.keys():
        probs[key]=counter_dic[key]/N
    return probs

In [32]:
probs=get_prob(counter_dic)

Implementing Edits

In [81]:
def insert_text(word):
    '''
    Input:
        word: the input string/word 
    Output:
        inserts: a set of all possible strings with one new letter inserted at every offset
    '''
    letters='abcdefghijklmnopqrstuvwxyz'
    splits=[(word[:i] , word[i:]) for i in range(len(word)+1)]
    inserts=[a+c+b for a,b in splits for c in letters]
    return inserts

In [82]:
def delete_text(word):
    '''
    Input:
        word: the string/word for which you will generate all possible words 
                in the vocabulary which have 1 missing character
    Output:
        deletes: a list of all possible strings obtained by deleting 1 character from word
    '''
    splits=[(word[:i],word[i:]) for i in range(len(word)+1)]
    deletes=[a+b[1:] for a,b in splits]
    return deletes

In [83]:
def switch_text(word):
    '''
    Input:
        word: input string
     Output:
        switches: a list of all possible strings with one adjacent charater switched
    '''
    splits=[(word[:i],word[i:]) for i in range(len(word)+1)]
    switches=[a+b[1]+b[0]+b[2:] for a,b in splits if len(b)>1]
    return switches

In [84]:
def replace_text(word):
    '''
    Input:
        word: the input string/word 
    Output:
        replaces: a list of all possible strings where we replaced one letter from the original word. 
    ''' 
    letters='abcdefghijklmnopqrstuvwxyz'
    splits=[(word[:i],word[i:]) for i in range(len(word)+1)]
    replaces=[a+c+b[1:] for a,b in splits for c in letters]
    return replaces

In [85]:
def edit1(word):
    """
    Input:
        word: the string/word for which we will generate all possible wordsthat are one edit away.
    Output:
        edit1: a set of words with one possible edit. Please return a set. and not a list.
    """
    edit1=set()
    edit1.update(insert_text(word))
    edit1.update(delete_text(word))
    edit1.update(switch_text(word))
    edit1.update(replace_text(word))
    return edit1

In [86]:
def edit2(word):
    '''
    Input:
        word: the input string/word 
    Output:
        edit2: a set of strings with all possible two edits
    '''
    edit2=set()
    words=edit1(word)
    for w in words:
        temp=edit1(w)
        edit2.update(temp)
    return edit2

In [79]:
def correction(word ,probs=probs , VOCAB=VOCAB, n=1):
    '''
    Input: 
        word: a user entered string to check for suggestions
        probs: a dictionary that maps each word to its probability in the corpus
        VOCAB: a set containing all the vocabulary
        n: number of possible word corrections you want returned in the dictionary
    Output: 
        n_best: a list of tuples with the most probable n corrected words and their probabilities.
    '''
    suggestions=[]
    n_best=[]
    suggestions=list((word in VOCAB and word) or (edit1(word)).intersection(VOCAB) or (edit2(word)).intersection(VOCAB) or word )
    print("suggestions = ", suggestions)
    n_best=[[s,probs[s]] for s in sorted(suggestions, key=lambda x: probs[x] , reverse=True)[:n]]
    return n_best

In [75]:
final_word=correction('arpeciation')

suggestions =  ['apreciation']


In [78]:
print(f'The autocorrected word is "{final_word[0][0]}" with probability of {final_word[0][1]}')

The autocorrected word is "apreciation" with probability of 1.7234827104523195e-06
